In [13]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

from os import listdir
from os.path import isfile, join
#set up working dir

import sys
sys.path.extend([
    '../tysserand/tysserand',
    '../mosna',
])

import seaborn as sns
from time import time
import copy
from skimage import color
import matplotlib as mpl
import napari

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform

import umap
# if not installed run: conda install -c conda-forge umap-learn
import hdbscan
from sklearn.cluster import OPTICS, cluster_optics_dbscan

from skimage import io

from scipy.stats import ttest_ind    # Welch's t-test
from scipy.stats import mannwhitneyu # Mann-Whitney rank test
from scipy.stats import ks_2samp     # Kolmogorov-Smirnov statistic

sys.path.append("/home/mouneem/mosna/")


from tysserand import tysserand as ty
from mosna import mosna






data_dir = Path("/home/mouneem/tysserand/")
path = str("/home/mouneem/tysserand/CORDELIER_PIERRE/")
path_nets = str("/home/mouneem/tysserand/nets/")
#path = str("/home/mouneem/tysserand/data/")

# select all CSVs
csv_path = "/home/mouneem/tysserand/CORDELIER_PIERRE/csv_M1_M2/"
csv_levels = [f for f in listdir(csv_path) if isfile(join(csv_path, f))]


save_dir = Path("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output/")
mosna_output = str("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output/")

sns.set_context('talk')



In [14]:
print(csv_levels)

['NVA_21-003.IMMCORE.C1v1_15T034275-11-Z1-2-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C2v1_15T010760-09-Z1-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C1v1_17T054628-06-Z1-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C2v1_15T008187-08-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C1v1_15T008187-08-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_16T058343-12-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_18T009975-12-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C2v1_15T027584-13-2-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C2v1_19T037195-08-Z1-2-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C1v1_15T027584-13-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_17T003826-06-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_16T037314-11-Z1-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C2v1_16T037314-11-Z3-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C2v1_19T002076-07-Z2-ImvessC2-4321.csv', 'NVA_21-003.IMMCORE.C1v1_16T042639-11-Z2-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_16T013087-8-ImvessC1-4518.csv', 'NVA_21-003.IMMCORE.C1v1_17T020895-03-Z2-I

In [26]:
i = 1
for csv_file in csv_levels[92::1]:
    print(i, csv_file)
    i += 1
    nodes = pd.read_csv(csv_path + csv_file , header = None)
    nodes.columns = ['id','x','y','class']

    ratio = 1
    nodes = nodes.sample(n = round(len(nodes) / ratio) )
    coords = nodes.loc[:,['x','y']].values


    #make colors
    nodes['color'] = '#7f8c8d'
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#e74c3c' #RED
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#e67e22' #ORANGE
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#f1c40f' #YELLOW
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#8e44ad' #PURPLE
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#2980b9' #LIGHT BLUE 
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#16a085' #GREEN SEA
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#2c3e50' #BLUE MIDNIGHT
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#27ae60' #GREEN
    nodes.loc[nodes['class'] == 'C1' , 'color' ] = '#7f8c8d' #GRAY


    cropped_coords = nodes.loc[:,['x','y']].values
    cropped_colors = nodes['color']

    ##########
    cropped_coords[:,0]  = (cropped_coords[:,0] * 0.1).round()
    cropped_coords[:,1]  = (cropped_coords[:,1] * 0.1).round()


    cropped_pairs = ty.build_delaunay(cropped_coords)


    pd.DataFrame(cropped_pairs).to_csv("/home/mouneem/tysserand/PanCK/output/pairs."+csv_file)

    pd.DataFrame(cropped_coords).to_csv("/home/mouneem/tysserand/PanCK/output/coords."+csv_file)
    pd.DataFrame(cropped_colors).to_csv("/home/mouneem/tysserand/PanCK/output/colors."+csv_file)

    nodes.loc[nodes['class'] == 'M1' , 'class' ] = 'M1' 
    nodes.loc[nodes['class'] == 'M2' , 'class' ] = 'M2' 
    nodes.loc[nodes['class'] == 'Other' , 'class' ] = 'Other' 
    nodes.to_csv("/home/mouneem/tysserand/PanCK/output/nodes."+csv_file)




    attributes = nodes['class']
    mosna_nodes = nodes
    edges = pd.DataFrame(data=cropped_pairs, columns=['source', 'target'])
    mosna_nodes = pd.DataFrame(data=cropped_coords, columns=['x', 'y'])
    mosna_nodes['Group'] = attributes.values
    mosna_nodes = mosna_nodes.join(pd.get_dummies(mosna_nodes['Group']))
    att = mosna_nodes['Group']
    attributes = mosna_nodes['Group'].unique()
    mosna_nodes.head()

    pd.DataFrame(attributes).to_csv("/home/mouneem/tysserand/PanCK/output/attributes."+csv_file)
    pd.DataFrame(edges).to_csv("/home/mouneem/tysserand/PanCK/output/edges."+csv_file)
    mosna_nodes.to_csv("/home/mouneem/tysserand/PanCK/output/mosna_nodes."+csv_file)


    attributes = nodes['class']
    edges = pd.DataFrame(data=cropped_pairs, columns=['source', 'target'])
    nodes = pd.DataFrame(data=cropped_coords, columns=['x', 'y'])
    nodes['Group'] = attributes.values
    nodes = nodes.join(pd.get_dummies(nodes['Group']))
    att = nodes['Group']
    attributes = nodes['Group'].unique()
    nodes.head()



    # Network statistics
    mixmat = mosna.mixing_matrix(nodes, edges, attributes)
    assort = mosna.attribute_ac(mixmat)
    print(type(mixmat))
    print(type(assort))
    # ------ Randomization ------
    mixmat_rand, assort_rand = mosna.randomized_mixmat(nodes, edges, attributes, n_shuffle=200, parallel=False)

    mixmat_mean, mixmat_std, mixmat_zscore = mosna.zscore(mixmat, mixmat_rand, return_stats=True)
    assort_mean, assort_std, assort_zscore = mosna.zscore(assort, assort_rand, return_stats=True)

    mixmat = mosna.mixmat_to_df(mixmat, attributes)
    mixmat_zscore = mosna.mixmat_to_df(mixmat_zscore, attributes)

    pd.DataFrame(mixmat).to_csv("/home/mouneem/tysserand/PanCK/output/mixmat."+csv_file)
    pd.DataFrame(mixmat_zscore).to_csv("/home/mouneem/tysserand/PanCK/output/mixmat_zscore."+csv_file)


1 NVA_21-003.IMMCORE.C1v1_15T010441-02-ImvessC1-4518.csv
<class 'numpy.ndarray'>
<class 'float'>


randomization: 100%|██████████████████████████| 200/200 [30:55<00:00,  9.28s/it]


2 NVA_21-003.IMMCORE.C1v1_17T006383-09-Z1-ImvessC1-4518.csv
<class 'numpy.ndarray'>
<class 'float'>


randomization: 100%|██████████████████████████| 200/200 [13:54<00:00,  4.17s/it]


3 NVA_21-003.IMMCORE.C2v1_16T034668-33-ImvessC2-4321.csv
<class 'numpy.ndarray'>
<class 'float'>


...

randomization: 100%|██████████████████████████| 200/200 [17:41<00:00,  5.31s/it]


55 NVA_21-003.IMMCORE.C2v1_18T047929-09-Z3-ImvessC2-4321.czi_80287_ObjData.fcs
<class 'numpy.ndarray'>
<class 'float'>


randomization: 100%|██████████████████████████| 200/200 [18:14<00:00,  5.47s/it]


56 NVA_21-003.IMMCORE.C1v1_17T048267-11-Z3-ImvessC1-4518.csv
<class 'numpy.ndarray'>
<class 'float'>


randomization: 100%|██████████████████████████| 200/200 [09:12<00:00,  2.76s/it]


In [ ]:



nodes = nodesAll
nodes.loc[nodes['class'] == 'C1' , 'class' ] = 'PanCK+CD3-CD8-CD20-'
nodes.loc[nodes['class'] == 'C2' , 'class' ] = 'PanCK-CD3+CD8+' 
nodes.loc[nodes['class'] == 'C3' , 'class' ] = 'PanCK-CD3+CD8-' 
nodes.loc[nodes['class'] == 'C4' , 'class' ] = 'PanCK-CD3-CD20+' 
nodes.loc[nodes['class'] == 'C5' , 'class' ] = 'PanCK-CD3+CD20+' 

attributes = nodes['class']

edges = pd.DataFrame(data=cropped_pairs, columns=['source', 'target'])
nodes = pd.DataFrame(data=cropped_coords, columns=['x', 'y'])
nodes['Group'] = attributes.values
nodes = nodes.join(pd.get_dummies(nodes['Group']))
att = nodes['Group']
attributes = nodes['Group'].unique()
nodes.head()



# Network statistics
mixmat = mosna.mixing_matrix(nodes, edges, attributes)
assort = mosna.attribute_ac(mixmat)

# ------ Randomization ------
mixmat_rand, assort_rand = mosna.randomized_mixmat(nodes, edges, attributes, n_shuffle=200, parallel=False)
mixmat_mean, mixmat_std, mixmat_zscore = mosna.zscore(mixmat, mixmat_rand, return_stats=True)
assort_mean, assort_std, assort_zscore = mosna.zscore(assort, assort_rand, return_stats=True)

mixmat = mosna.mixmat_to_df(mixmat, attributes)
mixmat_zscore = mosna.mixmat_to_df(mixmat_zscore, attributes)




In [ ]:
save_dir = Path("/home/abdelmounimessabbar/Bureau/PanCKmosna_output")

sns.set_context('talk')

title = "Assortativity by cell types:"
print(title)
fig, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(mixmat, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
plt.xticks(rotation=30, ha='right');
# plt.xticks(rotation=30, ha='right', fontsize=20);
# plt.yticks(fontsize=20);
plt.savefig(save_dir / "assortativity", bbox_inches='tight', facecolor='white')

print("Z-scored assortativity by cell types: {}".format(assort_zscore))
fig, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(mixmat_zscore, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
plt.xticks(rotation=30, ha='right');
plt.savefig(save_dir / "assortativity_z-scored", bbox_inches='tight', facecolor='white')